# Defining and Simulating Populations
### Vijay Ivaturi, Chris Rackauckas
### February 10, 2019

In [ ]:
using Pumas, DataFrames, LinearAlgebra, Plots

# Introduction

In this tutorial, we will cover the fundamentals of generating populations
to simulate with Pumas. We will demonstrate how to specify dosage regimens and
covariates, and then how to piece these together to form a population to
simulate.

## The model

Below is a Pumas model that specifies a 1-compartment oral absorption system
with between-subject variability on all the parameters. Details of the model
specification are provided in the introduction tutorial.

In [ ]:
model = @model begin
  @param begin
    θ ∈ VectorDomain(4)
    Ω ∈ PSDDomain(3)
    σ_prop ∈ RealDomain(init=0.1)
  end

  @random begin
    η ~ MvNormal(Ω)
  end

  @covariates isPM Wt

  @pre begin
    TVCL = isPM == 1 ? θ[1] : θ[4]
    CL = θ[1]*(Wt/70)^0.75*exp(η[1])
    V = θ[2]*(Wt/70)^0.75*exp(η[2])
    Ka = θ[3]*exp(η[3])
  end

  @dynamics begin
    Depot'   = -Ka*Depot
    Central' =  Ka*Depot - Central*CL/V
  end

  @vars begin
    conc = Central/V
  end

  @derived begin
    dv ~ @.Normal(conc,sqrt(conc^2*σ_prop+ eps()))
  end

end

## Setting up parameters

Next we provide the initial estimates of the parameters to simulate from. The fixed effects are provided
in the θ vector (CL, V, Ka) and the between-subject variability parameteres are provided in the Ω vector as
variances. So, 0.04 variance on Ω_11 suggests a 20% coefficient of variation. Similarly, σ_prop has a 20%
proportional residual error.

In [ ]:
fixeffs = (
  θ = [0.4,20,1.1,2],
  Ω = diagm(0 => [0.04,0.04,0.04]),
  σ_prop = 0.04
  )

## Single dose example

`DosageRegimen()` is the function that lets you construct a dosing regimen.
The first argument of the `DosageRegimen` is `amt` and is not a named argument.
All subsequent arguments need to be named. Lets try a simple example where you
provide a 100 mg dose at `time=0`.

In [ ]:
ev = DosageRegimen(100, time=0)
first(ev.data)

As you can see above, we provided a single 100 mg dose. `DosageRegimen` provides some defaults when
it creates the dataset, `time=0`, `evid=1`, `cmt=1`, `rate=0`, `ii=0` & `addl=0`. We can also provide
units to the `amt` and any other variable that is derived from `amt`, e.g. `rate`, will have associated units.
Handling of units will be covered in a different tutorial.

Note that `ev` is of type `DosageRegimen`. Specified like above,
`DosageRegimen` is one of the four fundamental building block of a `Subject` (more on `Subject` below).

### Building Subjects

Let's create a single subject

In [ ]:
s1 = Subject(id=1,evs=ev,cvs=(isPM=0, Wt=70))
for fn in fieldnames(Subject)
           x = getproperty(s1, fn)
           if !isa(x, Nothing)
               println(fn)
               println(x)
           end
end

Note that each `Subject` is an individual composed of:

  - `id`: an unique identifier
  - `obs`: observations, represented by `Pumas.Observation[]`
  - `cvs`: covariates
  - `evs`: events, represented by `Pumas.Event[]`

In the example above, we only provided the `id`, `evs`, and the `cvs`. Since  `obs` were not
provided, they are represented by an empty array. Lets take a closer at the events for this
subject 1.

In [ ]:
s1.events

The events are presented by basic information such as the dose of drug and associated units if specified, the time of dose administration,
the compartment number for administration and whether the dose is an instantaneous input or an infusion.

Below is how the covariates are represented

In [ ]:
s1.covariates

(Note: defining distributions for covariates will be discussed in detail later.)

Using this one subject, `s1`, let us simulate a simple concentration time
profile using the model above:

In [ ]:
obs = simobs(model,s1,fixeffs,obstimes=0:0.1:120)
plot(obs)

### Building Populations

Now, lets create one more subject, `s2`.

In [ ]:
s2 = Subject(id=2,evs=ev,cvs=(isPM=1,Wt=70))

If we want to simulate both `s1` and `s2` together, we need to bring these subjects together to form a `Population`. A `Population` is
essentially a collection of subjects.

In [ ]:
twosubjs =  Population([s1,s2])

Let's see the details of the first and the second subject

In [ ]:
twosubjs[1]

In [ ]:
twosubjs[2]

Now, we can simulate this `Population` of 2 subjects as below

In [ ]:
obs = simobs(model,twosubjs,fixeffs,obstimes=0:0.1:120)

When using `simobs` on more than one subject, i.e., on a `Population`, the
simulation is automatically parallelized across the subejcts.

In [ ]:
plot(obs)

Similarly, we can build a population of any number of subjects. But before we
do that, let's dive into covariate generation.

### Covariates

As was discussed earlier, a `Subject` can also be provided details regarding
covariates. In the model above, there are two covariates, `isPM` which stands
for _is the subject a poor metabolizer_ and takes a boolean of _yes_ and  _no_.
The second covariate is a continuous covariate where body weight `Wt` impacts
both `CL` and `V`. Let us now specify covariates to a population of 10 subjects.

In [ ]:
choose_covariates() = (isPM = rand([1, 0]),
                    Wt = rand(55:80))

`choose_covariates` will randomly choose a `isPM` and an `Wt` between 55-80 kgs

We can make a list with covariates for ten subjects through a list comprehension

In [ ]:
cvs = [ choose_covariates() for i in 1:10 ]
DataFrame(cvs)

Now, we add these covariates to the population as below. The `map(f,xs)` will
return the result of `f` on each element of `xs`. Let's map a function that
build's a subject with the randomly chosen covariates in order to build a
population:

In [ ]:
pop_with_covariates = Population(map(i -> Subject(id=i,evs=ev,cvs=choose_covariates()),1:10))

Simulate into the population

In [ ]:
obs = simobs(model,pop_with_covariates,fixeffs,obstimes=0:0.1:120)

and visualize the output

In [ ]:
plot(obs)

## Multiple dose example

The additional dosage regimen controls of the NMTRAN format are available
in `DosageRegimen`. For example, `ii` defines the "interdose interval", or the
time distance between two doses, while `addl` defines how many additional times
to repeat a dose. Thus, let's define a dose of 100 that's repeated 7 times
at 24 hour intervals:

In [ ]:
md =  DosageRegimen(100,ii=24,addl=6)

Let's create a new subject, `s3` with this dosage regimen:

In [ ]:
s3 = Subject(id=3,evs=md, cvs=(isPM=0,Wt=70))

and see the results:

In [ ]:
obs = simobs(model, s3, fixeffs,obstimes=0:0.1:240)
plot(obs)

## Combining dosage regimens

We can also combine dosage regimens to build a more complex regimen. Recall
from the introduction that using arrays will build the element-wise combinations.
Thus let's build a dose of 500 into compartment 1 at time 0, and 7 doses into
compartment 1 of 100 spaced by 24 hours:

In [ ]:
ldmd = DosageRegimen([500,100],cmt=1, time=[0,24], addl=[0,6],ii=[0,24])

Let's see if this result matches our intuition:

In [ ]:
s4 = Subject(id=4, evs=ldmd, cvs=(isPM=0,Wt=70))
obs = simobs(model, s4, fixeffs,obstimes=0:0.1:120)
plot(obs, ylims=(0,50))

Another way to build complex dosage regiments is to combine previously constructed
regimens into a single regimen. For example:

In [ ]:
e1 = DosageRegimen(500,cmt=1, time=0, addl=0,ii=0)
e2 = DosageRegimen(100,cmt=1, time=24, addl=6,ii=24)
evs = DosageRegimen(e1,e2)
obs = simobs(model, s4, fixeffs,obstimes=0:0.1:120)
plot(obs, ylims=(0,50))

is the same regimen as before.

Putting these ideas together, we can define a population where individuals
with different covariates undergo different regimens, and simulate them
all together with automatic parallelism:

In [ ]:
e1 = DosageRegimen(100, ii=24, addl=6)
e2 = DosageRegimen(50,  ii=12, addl=13)
e3 = DosageRegimen(200, ii=24, addl=2)

In [ ]:
pop1 = Population(map(i -> Subject(id=i,evs=e1,cvs=choose_covariates()),1:5))
pop2 = Population(map(i -> Subject(id=i,evs=e2,cvs=choose_covariates()),6:8))
pop3 = Population(map(i -> Subject(id=i,evs=e3,cvs=choose_covariates()),9:10))
pop = Population(vcat(pop1,pop2,pop3))

In [ ]:
obs = simobs(model,pop,fixeffs,obstimes=0:0.1:120)
plot(obs)

## Defining Infusions

As specified in the NMTRAN format, an infusion is a dosage which is defined
as having a non-zero positive rate at which the drug enters the system. Let's
define a single infusion dose of total amount 100 with a rate of 3 into the
second compartment:

In [ ]:
inf = DosageRegimen(100, rate=3, cmt=2)

Now let's simulate a subject undergoing this treatment strategy:

In [ ]:
s5 = Subject(id=5, evs=inf, cvs=(isPM=0,Wt=70))
obs = simobs(model, s5, fixeffs, obstimes=0:0.1:120)
plot(obs)

## Final Note on Julia Programming

Note that all of these functions are standard Julia functions, and thus standard
Julia programming constructions can be utilized to simplify the construction of
large populations. We already demonstrated the use of `map` and a comprehension,
but we can also make use of constructs like for loops.

## Conclusion

This tutorial shows the tools for generating populations of infinite complexity,
defining covariates and dosage regimens on the fly and simulating the results
of the model.